In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [12]:
data = pd.read_csv('../data/tweets_with_labels_no_depression.csv')
tweets, label = data["tweets"], data["labels"]

In [13]:
dataset = tf.data.Dataset.from_tensor_slices((tweets, label))

In [14]:
max_tokens = tweets.str.split().str.len().max() + 1

In [15]:
tweets_vectorizer = tf.keras.layers.TextVectorization(
        max_tokens=max_tokens)

tweets_vectorizer.adapt(tweets)

model = tf.keras.Sequential([
    tweets_vectorizer,
    tf.keras.layers.Embedding(max_tokens, 64, mask_zero=True),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [16]:
shuffle_dataset = dataset.shuffle(10_000)

train_dataset = shuffle_dataset.take(7_200)
validation_dataset = shuffle_dataset.skip(7200).take(800)
test_dataset = shuffle_dataset.skip(8000).take(2000)

In [17]:
cached_train = train_dataset.shuffle(1024).batch(1024).cache()
cached_validation = validation_dataset.batch(1024).cache()
cached_test = test_dataset.batch(512).cache()

In [18]:
history = model.fit(cached_train, epochs=30, validation_data=cached_validation)

Epoch 1/30
8/8 [==============================] - 1s 28ms/step - loss: 0.6820 - accuracy: 0.6542 - val_loss: 0.6509 - val_accuracy: 0.8000
Epoch 2/30
8/8 [==============================] - 0s 11ms/step - loss: 0.6407 - accuracy: 0.7743 - val_loss: 0.6085 - val_accuracy: 0.8000
Epoch 3/30
8/8 [==============================] - 0s 11ms/step - loss: 0.6041 - accuracy: 0.7743 - val_loss: 0.5690 - val_accuracy: 0.8000
Epoch 4/30
8/8 [==============================] - 0s 11ms/step - loss: 0.5712 - accuracy: 0.7743 - val_loss: 0.5337 - val_accuracy: 0.8000
Epoch 5/30
8/8 [==============================] - 0s 11ms/step - loss: 0.5443 - accuracy: 0.7743 - val_loss: 0.5065 - val_accuracy: 0.8000
Epoch 6/30
8/8 [==============================] - 0s 11ms/step - loss: 0.5265 - accuracy: 0.7743 - val_loss: 0.4894 - val_accuracy: 0.8000
Epoch 7/30
8/8 [==============================] - 0s 11ms/step - loss: 0.5170 - accuracy: 0.7743 - val_loss: 0.4800 - val_accuracy: 0.8000
Epoch 8/30
8/8 [===========

In [19]:
model.evaluate(cached_test)

4/4 [==============================] - 0s 5ms/step - loss: 0.3860 - accuracy: 0.8260


[0.38600799441337585, 0.8259999752044678]

In [33]:
model.predict(["I don't like Dream from minecraft manhunt"])

array([[0.6541928]], dtype=float32)

In [15]:
label.value_counts()
# there 2314 depressed tweets
# the word 'depression' show up 2579 times

0    8000
1    2314
Name: labels, dtype: int64